# Using Naive Bayes algorithm for spam detection

In this assigment, you will predict if a sms message is 'spam' or 'ham' (i.e. not 'spam') using the Bernoulli Naive Bayes *classifier*.

The training data is from the UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection.  Please download the zip file before running the code below. 


## Step 1:  Getting, understanding, and cleaning the dataset


###  Importing the dataset

In [1]:
# Import the usual libraries
import numpy as np 
import pandas as pd  # To read in the dataset we will use the Panda's library
df = pd.read_table('SMSSpamCollection', sep = '\t', header=None, names=['label', 'sms_message'])

# Next we observe the first 5 rows of the data to ensure everything was read correctly
df.head() 

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### Data preprocesssing
It would be more convenient if the labels were binary instead of 'ham' and 'spam'.  This way our code can always work with numerical values instead of strings.

In [2]:
df['label']=df.label.map({'spam':1, 'ham':0})
df.head() # Again, lets observe the first 5 rows to make sure everything worked before we continue

,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


### Splitting the dcoument into training set and test set

In [3]:
# This time we will use sklearn's method for seperating the data
from sklearn.model_selection import train_test_split

df_train_msgs, df_test_msgs, df_ytrain, df_ytest = train_test_split(df['sms_message'],df['label'], random_state=0)

#Looking at the train/test split
print("The number of training examples: ", df_train_msgs.shape[0])
print("The number of test exampels: ", df_test_msgs.shape[0])

print("The first four labels")
print(df_ytrain[0:4])

print("The first four sms messages")
print(df_train_msgs[0:4])


The number of training examples:  4179
The number of test exampels:  1393
The first four labels
872     0
831     1
1273    0
3314    0
Name: label, dtype: int64
The first four sms messages
872     Its going good...no problem..but still need li...
831     U have a secret admirer. REVEAL who thinks U R...
1273                                                Ok...
3314    Huh... Hyde park not in mel ah, opps, got conf...
Name: sms_message, dtype: object


###  Creating the feature vector from the text (feature extraction)

Each message will have its own feature vector.  For each message we will create its feature vector as we discussed in class; we will have a feature for every word in our vocabulary.  The $j$th feature is set to one ($x_j=1$) if the $j$th word from our vocabulary occurs in the message, and set the $j$ feature is set to $0$ otherwise ($x_j=0$).

We will use the sklearn method CountVectorize to create the feature vectors for every messge.

We could have written the code to do this directly by performing the following steps:
* remove capitalization
* remove punctuation 
* tokenize (i.e. split the document into individual words)
* count frequencies of each token 
* remove 'stop words' (these are words that will not help us predict since they occur in most documents, e.g. 'a', 'and', 'the', 'him', 'is' ...

In [4]:
# importing the library
from sklearn.feature_extraction.text import CountVectorizer
# creating an instance of CountVectorizer
# Note there are issues with the way CountVectorizer removes stop words.  To learn more: https://scikit-learn.org/stable/modules/feature_extraction.html#stop-words
vectorizer = CountVectorizer(binary = True, stop_words='english')

# If we wanted to perform Multnomial Naive Bayes, we would include the word counts and use the following code
#vectorizer = CountVectorizer(stop_words='english')

# To see the 'count_vector' object
print(vectorizer)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


In [5]:
# To see the 'stop words' 
#print(vectorizer.get_stop_words())

In [6]:
# Create the vocabulary for our feature transformation
vectorizer.fit(df_train_msgs)

# Next we create the feature vectors for both the training data and the test data
X_train = vectorizer.transform(df_train_msgs).toarray() # code to turn the training emails into a feature vector
X_test = vectorizer.transform(df_test_msgs).toarray() # code to turn the test email into a feature vector

# Changing the target vectors data type  
y_train=df_ytrain.to_numpy() # Convereting from a Panda series to a numpy array
y_test = df_ytest.to_numpy()

# To observe what the data looks like 
print("The label of the first training example: ", y_train[0])
print("The first training example: ", X_train[0].tolist())# I needed to covernt the datatype to list so all the feature values would be printed

The label of the first training example:  0
The first training example:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Your code goes here

In [7]:
# You should NOT use the features of sklearn library in your code.
#### TO-DO #####

# seperate the data into 2 parts
X_train_0 = X_train[y_train == 0]
X_train_1 = X_train[y_train == 1]

p_y0 = X_train_0.shape[0] / y_train.shape[0]
p_y1 = X_train_1.shape[0] / y_train.shape[0]

print("The estimated value of P(y) for y = 1 is: ", p_y1)
print("The estimated value of P(y) for y = 0 is: ", p_y0)

The estimated value of P(y) for y = 1 is:  0.13448193347690834
The estimated value of P(y) for y = 0 is:  0.8655180665230916


In [8]:
# {index of feature: phi0_feature or phi1_feature}
phi0_vocabulary = {}
phi1_vocabulary = {}

def updatePhiVocabulary(m=0.):
    for feature in vectorizer.vocabulary_:
        index = int(vectorizer.vocabulary_[feature])
        # each index has only 2 possible values: 0 and 1
        phi0_vocabulary[index] = (np.count_nonzero(X_train_0[:, [index]] == 1) + m) / (X_train_0.shape[0] + m * 2)
        phi1_vocabulary[index] = (np.count_nonzero(X_train_1[:, [index]] == 1) + m) / (X_train_1.shape[0] + m * 2)

updatePhiVocabulary()

# save to a json file
import json
json0 = json.dumps(phi0_vocabulary)
json1 = json.dumps(phi1_vocabulary)
f = open("phi0_vocabulary.json","w")
f.write(json0)
f = open("phi1_vocabulary.json","w")
f.write(json1)
f.close()

In [9]:
index_admirer = vectorizer.vocabulary_['admirer']
print("The estimated value of phi_admirer for y = 1 is: ", phi1_vocabulary[index_admirer])
print("The estimated value of phi_admirer for y = 0 is: ", phi0_vocabulary[index_admirer])
print()
index_secret = vectorizer.vocabulary_['secret']
print("The estimated value of phi_secret for y = 1 is: ", phi1_vocabulary[index_secret])
print("The estimated value of phi_secret for y = 0 is: ", phi0_vocabulary[index_secret])

The estimated value of phi_admirer for y = 1 is:  0.014234875444839857
The estimated value of phi_admirer for y = 0 is:  0.0

The estimated value of phi_secret for y = 1 is:  0.014234875444839857
The estimated value of phi_secret for y = 0 is:  0.000552944429084877


In [10]:
def predictSingle(x):
    px_y0, px_y1 = 0., 0.
    p_y0 = X_train_0.shape[0] / y_train.shape[0]
    p_y1 = X_train_1.shape[0] / y_train.shape[0]
    
    for i in range(len(x)):
        phi_feature_0 = phi0_vocabulary[i] if x[i] else 1 - phi0_vocabulary[i]
        phi_feature_1 = phi1_vocabulary[i] if x[i] else 1 - phi1_vocabulary[i]   
        px_y0 += np.log(phi_feature_0)
        px_y1 += np.log(phi_feature_1)
    
    px_y0 += np.log(p_y0)
    px_y1 += np.log(p_y1)
    
    return 0 if px_y0 > px_y1 else 1

# Predict
def predictGroup(X):
    y_predict = []
    error = 0
    for i in range(X.shape[0]):
        temp_result = predictSingle(X[i])
        y_predict += [temp_result]
        if temp_result != y_test[i]:
            error += 1
    error_rate = error / X_test.shape[0]
    return y_predict, error_rate, error
    
y_predict, error_rate, error = predictGroup(X_test)
print("The classes which were predicted for the first 5 examples in the test set are: ", y_predict[0:5])
print()

print("The classes which were predicted for the last 5 examples in the test set are: ", y_predict[-5:])
print()

print("The classes which were predicted for the first 50 examples in the test set are: ")
print(y_predict[0:50])
print()

print("The percentage error on the examples in the test file is: ", error_rate)
print("The total number of test examples classified correctly is: ", X_test.shape[0] - error)
print("The total number of test examples classified incorrectly is: ", error)

E:\anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.
E:\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


The classes which were predicted for the first 5 examples in the test set are:  [0, 0, 0, 0, 0]

The classes which were predicted for the last 5 examples in the test set are:  [0, 1, 1, 0, 0]

The classes which were predicted for the first 50 examples in the test set are: 
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

The percentage error on the examples in the test file is:  0.048815506101938265
The total number of test examples classified correctly is:  1325
The total number of test examples classified incorrectly is:  68


In [11]:
# add smoothing
min_error_rate = 1
best_m = 0
best_y_predict = []
m = 0.
while m <= 1.:
    updatePhiVocabulary(m)
    y_predict, error_rate, error = predictGroup(X_test)
    if error_rate < min_error_rate:
        best_m, min_error_rate, best_y_predict = m, error_rate, y_predict[:]
    m += 0.05

print("When m = {}, the error rate is minimum: {}".format(best_m, min_error_rate))

E:\anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.
E:\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


When m = 0.05, the error rate is minimum: 0.013639626704953339


In [12]:
# using Zero-R
zero_r_label = 0 if X_train_0.shape[0] > X_train_1.shape[0] else 1
print("The label which Zero-R predicts is: ", zero_r_label)

error_rate = sum([zero_r_label ^ y_test[i] for i in range(X_test.shape[0])]) / X_test.shape[0]
print("The error rate of Zero-R is: ", error_rate)

The label which Zero-R predicts is:  0
The error rate of Zero-R is:  0.13280689160086145


In [13]:
##############